In [566]:
# Import necessary libraries
import pandas as pd
import numpy as np

from pathlib import Path
from sqlalchemy import create_engine, text
import sqlite3

## Data extraction from the data.xlsx file

In [490]:
# Read the data into a Pandas DataFrame
raw_data_df = pd.read_excel('data/data.xlsx')
raw_data_df.head()

,setting,date,source,indicator_abbr,indicator_name,dimension,subgroup,estimate,se,ci_lb,...,iso3,favourable_indicator,indicator_scale,ordered_dimension,subgroup_order,reference_subgroup,whoreg6,wbincome2024,dataset_id,update
0,Afghanistan,2004,NNS,overweight,Overweight prevalence in children aged < 5 yea...,Child's age (6 groups) (0-59m),0-5 months,NaN,NaN,NaN,...,AFG,0,100,1,1,0,Eastern Mediterranean,Low income,rep_nut,17 June 2024
1,Afghanistan,2004,NNS,overweight,Overweight prevalence in children aged < 5 yea...,Child's age (6 groups) (0-59m),12-23 months,4.3,NaN,NaN,...,AFG,0,100,1,3,0,Eastern Mediterranean,Low income,rep_nut,17 June 2024
2,Afghanistan,2004,NNS,overweight,Overweight prevalence in children aged < 5 yea...,Child's age (6 groups) (0-59m),24-35 months,3.0,NaN,NaN,...,AFG,0,100,1,4,0,Eastern Mediterranean,Low income,rep_nut,17 June 2024
3,Afghanistan,2004,NNS,overweight,Overweight prevalence in children aged < 5 yea...,Child's age (6 groups) (0-59m),36-47 months,5.6,NaN,NaN,...,AFG,0,100,1,5,0,Eastern Mediterranean,Low income,rep_nut,17 June 2024
4,Afghanistan,2004,NNS,overweight,Overweight prevalence in children aged < 5 yea...,Child's age (6 groups) (0-59m),48-59 months,6.4,NaN,NaN,...,AFG,0,100,1,6,0,Eastern Mediterranean,Low income,rep_nut,17 June 2024


In [541]:
# Get a brief summary of the raw_data DataFrame.
raw_data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 157095 entries, 0 to 157094
Data columns (total 24 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   setting               157095 non-null  object 
 1   date                  157095 non-null  int64  
 2   source                157095 non-null  object 
 3   indicator_abbr        157095 non-null  object 
 4   indicator_name        157095 non-null  object 
 5   dimension             157095 non-null  object 
 6   subgroup              157095 non-null  object 
 7   estimate              153334 non-null  float64
 8   se                    136679 non-null  float64
 9   ci_lb                 142754 non-null  float64
 10  ci_ub                 142754 non-null  float64
 11  population            147089 non-null  float64
 12  flag                  157095 non-null  object 
 13  setting_average       157095 non-null  float64
 14  iso3                  157095 non-null  object 
 15  

There are 157,095 records and 24 columns of raw data in our dataframe.

## Data cleaning

Check for any columns that are empty and drop them from the dataframe.

In [542]:
# Check for empty columns
empty_columns = raw_data_df.columns[raw_data_df.isnull().all()]
print("Empty columns:", empty_columns)

# Remove empty columns
cleaned_df = raw_data_df.dropna(axis=1, how='all')

# Verify data frame info
print("DataFrame info after removing empty columns:")
cleaned_df.info()

Empty columns: Index([], dtype='object')
DataFrame info after removing empty columns:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 157095 entries, 0 to 157094
Data columns (total 24 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   setting               157095 non-null  object 
 1   date                  157095 non-null  int64  
 2   source                157095 non-null  object 
 3   indicator_abbr        157095 non-null  object 
 4   indicator_name        157095 non-null  object 
 5   dimension             157095 non-null  object 
 6   subgroup              157095 non-null  object 
 7   estimate              153334 non-null  float64
 8   se                    136679 non-null  float64
 9   ci_lb                 142754 non-null  float64
 10  ci_ub                 142754 non-null  float64
 11  population            147089 non-null  float64
 12  flag                  157095 non-null  object 
 13  setting_average   

There are still 24 columns, so no empty columns were found.

Count the unique values for each column so that we can determine if any have a single value, we can drop them from the dataframe.  Such columns will not provide any meaningful insights.

In [543]:
# Count the number of unique values in each column
unique_counts = cleaned_df.nunique()

# Create a new DataFrame to store the counts
unique_counts_df = pd.DataFrame(unique_counts, columns=['Unique Count'])

# Display the new DataFrame
print("DataFrame with the count of unique values in each column:")
unique_counts_df

DataFrame with the count of unique values in each column:


,Unique Count
setting,156
date,34
source,19
indicator_abbr,15
indicator_name,15
dimension,6
subgroup,4517
estimate,130841
se,135364
ci_lb,136109


Columns 'favourable indicator', 'indicator_scale', 'dataset_id', and 'update' have single values so we can drop them.

In [544]:
# Remove columns with a single value from the DataFrame
cleaned_df = cleaned_df.drop(columns=['favourable_indicator', 'indicator_scale', 'dataset_id', 'update'])

# Display the updated DataFrame
print("DataFrame info after removing single value columns:\n")
cleaned_df.info()

DataFrame info after removing single value columns:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 157095 entries, 0 to 157094
Data columns (total 20 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   setting             157095 non-null  object 
 1   date                157095 non-null  int64  
 2   source              157095 non-null  object 
 3   indicator_abbr      157095 non-null  object 
 4   indicator_name      157095 non-null  object 
 5   dimension           157095 non-null  object 
 6   subgroup            157095 non-null  object 
 7   estimate            153334 non-null  float64
 8   se                  136679 non-null  float64
 9   ci_lb               142754 non-null  float64
 10  ci_ub               142754 non-null  float64
 11  population          147089 non-null  float64
 12  flag                157095 non-null  object 
 13  setting_average     157095 non-null  float64
 14  iso3                157095 non-

The four single value columns were correctly dropped.  We now have 20 columns in the dataframe.

1. The *flag* column includes notes, author, and reference title.
2. The *source* column includes a code for reprenting the source data type.
3. The meaning of the *reference_subgroup* column is unknown.
4. The *se* column refers to the standard error in the prevalence estimate.
5. The *ci_lb* and *ci_ub* columns refer to confidence intervals upper and lower bounds respectively.
6. The *ordered_dimension* column is a boolean flag that indicates if the *dimension* column is a nominal or ordinal variable.  In this dataset, *0* represents a nominal variable and *1* represents an ordinal variable.  Nominal variables are for the dimensions of *sex*, *place of residence*, and *subnational region*.  Ordinal variables are for the dimensions of *age*, *economic status*, and *education level*. Ordinal dimensions use the *subgroup_order* column to denote the specific ordering.
7. The *wbincome2024* column is an income group classification provided by The World Bank.  It is a classification of the subnational region dimension.  This is extra data since economic status is already a covered dimension.
8. The *indicator_name* column is a description for *indicator_abbr*.

None data is needed for our analysis, so we will drop it from the dataframe.

In [545]:
# Remove 'flag' column from the DataFrame
cleaned_df = cleaned_df.drop(columns=['flag', 'source', 'reference_subgroup', 'se', 'ci_lb', 'ci_ub', 'ordered_dimension', 'wbincome2024', 'indicator_name'])

# Display the updated DataFrame
print("DataFrame info after removing the unecessary columns:\n")
cleaned_df.info()

DataFrame info after removing the unecessary columns:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 157095 entries, 0 to 157094
Data columns (total 11 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   setting          157095 non-null  object 
 1   date             157095 non-null  int64  
 2   indicator_abbr   157095 non-null  object 
 3   dimension        157095 non-null  object 
 4   subgroup         157095 non-null  object 
 5   estimate         153334 non-null  float64
 6   population       147089 non-null  float64
 7   setting_average  157095 non-null  float64
 8   iso3             157095 non-null  object 
 9   subgroup_order   157095 non-null  int64  
 10  whoreg6          157095 non-null  object 
dtypes: float64(3), int64(2), object(6)
memory usage: 13.2+ MB


Rename the columns with more intuitive titles and reorder them.

In [546]:
# Rename the columns
cleaned_df = cleaned_df.rename(columns={
    'setting': 'Country',
    'date': 'Year',
    'indicator_abbr': 'Anthropometric Indicator',
    'dimension': 'Dimension',
    'subgroup': 'Dimension Value',
    'subgroup_order': 'Dimension Value Order',
    'setting_average': 'Country Avg',
    'iso3': 'Country ISO-3 Code',
    'whoreg6': 'Region',
    'estimate': 'Prevalence Estimate %',
    'population': 'Under-Five Population (million)'
})

# Reorder the columns
cleaned_df = cleaned_df[
    [
        'Region',
        'Country ISO-3 Code',
        'Country',
        'Year',
        'Dimension',
        'Dimension Value',
        'Dimension Value Order',
        'Anthropometric Indicator',
        'Prevalence Estimate %',
        'Under-Five Population (million)',
        'Country Avg'
    ]
]

# Check the DataFrame information
cleaned_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 157095 entries, 0 to 157094
Data columns (total 11 columns):
 #   Column                           Non-Null Count   Dtype  
---  ------                           --------------   -----  
 0   Region                           157095 non-null  object 
 1   Country ISO-3 Code               157095 non-null  object 
 2   Country                          157095 non-null  object 
 3   Year                             157095 non-null  int64  
 4   Dimension                        157095 non-null  object 
 5   Dimension Value                  157095 non-null  object 
 6   Dimension Value Order            157095 non-null  int64  
 7   Anthropometric Indicator         157095 non-null  object 
 8   Prevalence Estimate %            153334 non-null  float64
 9   Under-Five Population (million)  147089 non-null  float64
 10  Country Avg                      157095 non-null  float64
dtypes: float64(3), int64(2), object(6)
memory usage: 13.2+ MB


From the summary information, we see that columns Prevalence Estimate % and Under-Five Population (million) have empty values.  Those rows need to be removed from the dataframe.

In [547]:
# Remove rows where 'Under-Five Population (million)' and 'Prevalence Estimate %' are empty
cleaned_df = cleaned_df.dropna(subset=['Under-Five Population (million)', 'Prevalence Estimate %'])

# Check the DataFrame after dropping rows
cleaned_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 147089 entries, 1 to 157094
Data columns (total 11 columns):
 #   Column                           Non-Null Count   Dtype  
---  ------                           --------------   -----  
 0   Region                           147089 non-null  object 
 1   Country ISO-3 Code               147089 non-null  object 
 2   Country                          147089 non-null  object 
 3   Year                             147089 non-null  int64  
 4   Dimension                        147089 non-null  object 
 5   Dimension Value                  147089 non-null  object 
 6   Dimension Value Order            147089 non-null  int64  
 7   Anthropometric Indicator         147089 non-null  object 
 8   Prevalence Estimate %            147089 non-null  float64
 9   Under-Five Population (million)  147089 non-null  float64
 10  Country Avg                      147089 non-null  float64
dtypes: float64(3), int64(2), object(6)
memory usage: 13.5+ MB


The Child's Age dimension includes additional indicators that append M and F to the Anthropometric Indicator.  These entries are redundant to our analysis as we already have aggregate data for the dimension of Sex.  We remove these rows.

In [548]:
# Remove rows where 'Anthropometric Indicator' ends with '_F' or '_M'
cleaned_df = cleaned_df[~cleaned_df['Anthropometric Indicator'].str.endswith(('_F', '_M'))]

# Check the DataFrame after dropping rows
cleaned_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 100770 entries, 1 to 157082
Data columns (total 11 columns):
 #   Column                           Non-Null Count   Dtype  
---  ------                           --------------   -----  
 0   Region                           100770 non-null  object 
 1   Country ISO-3 Code               100770 non-null  object 
 2   Country                          100770 non-null  object 
 3   Year                             100770 non-null  int64  
 4   Dimension                        100770 non-null  object 
 5   Dimension Value                  100770 non-null  object 
 6   Dimension Value Order            100770 non-null  int64  
 7   Anthropometric Indicator         100770 non-null  object 
 8   Prevalence Estimate %            100770 non-null  float64
 9   Under-Five Population (million)  100770 non-null  float64
 10  Country Avg                      100770 non-null  float64
dtypes: float64(3), int64(2), object(6)
memory usage: 9.2+ MB


The Sub-national Region dimension value is irrelevant to our analysis and is also only tied to the World Bank Income Group column, so those rows will be removed.

In [549]:
# Remove entries where 'Dimension' is 'Subnational Region'
cleaned_df = cleaned_df[cleaned_df['Dimension'] != 'Subnational region']

# Check the DataFrame after dropping rows
cleaned_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 56618 entries, 1 to 157082
Data columns (total 11 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Region                           56618 non-null  object 
 1   Country ISO-3 Code               56618 non-null  object 
 2   Country                          56618 non-null  object 
 3   Year                             56618 non-null  int64  
 4   Dimension                        56618 non-null  object 
 5   Dimension Value                  56618 non-null  object 
 6   Dimension Value Order            56618 non-null  int64  
 7   Anthropometric Indicator         56618 non-null  object 
 8   Prevalence Estimate %            56618 non-null  float64
 9   Under-Five Population (million)  56618 non-null  float64
 10  Country Avg                      56618 non-null  float64
dtypes: float64(3), int64(2), object(6)
memory usage: 5.2+ MB


After removing unecessary columns and rows, as well as empty rows, we are left with 56,618 entries and 11 columns.

In [550]:
# Check the clean data
cleaned_df.head()

,Region,Country ISO-3 Code,Country,Year,Dimension,Dimension Value,Dimension Value Order,Anthropometric Indicator,Prevalence Estimate %,Under-Five Population (million),Country Avg
1,Eastern Mediterranean,AFG,Afghanistan,2004,Child's age (6 groups) (0-59m),12-23 months,3,overweight,4.3,189.0,4.6
2,Eastern Mediterranean,AFG,Afghanistan,2004,Child's age (6 groups) (0-59m),24-35 months,4,overweight,3.0,182.0,4.6
3,Eastern Mediterranean,AFG,Afghanistan,2004,Child's age (6 groups) (0-59m),36-47 months,5,overweight,5.6,227.0,4.6
4,Eastern Mediterranean,AFG,Afghanistan,2004,Child's age (6 groups) (0-59m),48-59 months,6,overweight,6.4,241.0,4.6
5,Eastern Mediterranean,AFG,Afghanistan,2004,Child's age (6 groups) (0-59m),6-11 months,2,overweight,1.9,107.0,4.6


In [551]:
# Apply proper capitalization to Anthropometric Indicator values
mapping = {
    'overweight': 'Overweight',
    'stunting': 'Stunting',
    'underweight': 'Underweight',
    'wasting': 'Wasting',
    'wastingsev': 'Wasting Severe'
}

# Map the values in the 'Anthropometric Indicator' column
cleaned_df['Anthropometric Indicator'] = cleaned_df['Anthropometric Indicator'].replace(mapping)

# Check the unique values in the 'Anthropometric Indicator' column
unique_values = cleaned_df['Anthropometric Indicator'].unique()
print(unique_values)

['Overweight' 'Stunting' 'Underweight' 'Wasting' 'Wasting Severe']


In [552]:
# Clean up dimension names using a dictionary
dimension_mapping = {
    "Child's age (6 groups) (0-59m)": 'Age (months)',
    "Education (3 groups)": "Education (mother)"
}

# Apply the mapping to the 'Dimension' column
cleaned_df['Dimension'] = cleaned_df['Dimension'].replace(dimension_mapping)

# Check the unique values in the column
unique_values = cleaned_df['Dimension'].unique()
print(unique_values)

['Age (months)' 'Sex' 'Place of residence'
 'Economic status (wealth quintile)' 'Education (mother)']


In [553]:
# Export cleaned_df as CSV file
cleaned_df.to_csv("data/clean_data.csv", index=False)

## Filtering of clean data by each of the dimensions and exporting to CSVs in preparation of data loading.

In [554]:
# Filter cleaned_df to dataframes with only rows relevant to each dimension
age_df = cleaned_df[cleaned_df['Dimension'] == 'Age (months)']
sex_df = cleaned_df[cleaned_df['Dimension'] == 'Sex']
residence_df = cleaned_df[cleaned_df['Dimension'] == 'Place of residence']
economic_status_df = cleaned_df[cleaned_df['Dimension'] == 'Economic status (wealth quintile)']
education_df = cleaned_df[cleaned_df['Dimension'] == 'Education (mother)']

In [555]:
# Check the new DataFrames
age_df.head()

,Region,Country ISO-3 Code,Country,Year,Dimension,Dimension Value,Dimension Value Order,Anthropometric Indicator,Prevalence Estimate %,Under-Five Population (million),Country Avg
1,Eastern Mediterranean,AFG,Afghanistan,2004,Age (months),12-23 months,3,Overweight,4.3,189.0,4.6
2,Eastern Mediterranean,AFG,Afghanistan,2004,Age (months),24-35 months,4,Overweight,3.0,182.0,4.6
3,Eastern Mediterranean,AFG,Afghanistan,2004,Age (months),36-47 months,5,Overweight,5.6,227.0,4.6
4,Eastern Mediterranean,AFG,Afghanistan,2004,Age (months),48-59 months,6,Overweight,6.4,241.0,4.6
5,Eastern Mediterranean,AFG,Afghanistan,2004,Age (months),6-11 months,2,Overweight,1.9,107.0,4.6


In [506]:
# Check the new DataFrames
sex_df.head()

,Region,Country ISO-3 Code,Country,Year,Dimension,Dimension Value,Dimension Value Order,Anthropometric Indicator,Prevalence Estimate %,Under-Five Population (million),Country Avg
6,Eastern Mediterranean,AFG,Afghanistan,2004,Sex,Female,0,Overweight,2.6,436.0,4.6
7,Eastern Mediterranean,AFG,Afghanistan,2004,Sex,Male,0,Overweight,6.3,510.0,4.6
26,Eastern Mediterranean,AFG,Afghanistan,2004,Sex,Female,0,Stunting,58.5,436.0,59.3
27,Eastern Mediterranean,AFG,Afghanistan,2004,Sex,Male,0,Stunting,60.0,510.0,59.3
46,Eastern Mediterranean,AFG,Afghanistan,2004,Sex,Female,0,Underweight,33.0,436.0,32.9


In [507]:
# Check the new DataFrames
residence_df.head()

,Region,Country ISO-3 Code,Country,Year,Dimension,Dimension Value,Dimension Value Order,Anthropometric Indicator,Prevalence Estimate %,Under-Five Population (million),Country Avg
106,Eastern Mediterranean,AFG,Afghanistan,2013,Place of residence,Rural,0,Overweight,5.969128,3.234018e+06,5.341543
107,Eastern Mediterranean,AFG,Afghanistan,2013,Place of residence,Urban,0,Overweight,3.338518,1.013278e+06,5.341543
162,Eastern Mediterranean,AFG,Afghanistan,2013,Place of residence,Rural,0,Stunting,43.330780,3.214813e+06,40.413544
163,Eastern Mediterranean,AFG,Afghanistan,2013,Place of residence,Urban,0,Stunting,31.130192,1.010235e+06,40.413544
218,Eastern Mediterranean,AFG,Afghanistan,2013,Place of residence,Rural,0,Underweight,26.375696,3.373931e+06,24.643473


In [508]:
# Check the new DataFrames
economic_status_df.head()

,Region,Country ISO-3 Code,Country,Year,Dimension,Dimension Value,Dimension Value Order,Anthropometric Indicator,Prevalence Estimate %,Under-Five Population (million),Country Avg
386,Eastern Mediterranean,AFG,Afghanistan,2018,Economic status (wealth quintile),Quintile 1 (poorest),1,Overweight,4.486930,3576.301514,4.112625
387,Eastern Mediterranean,AFG,Afghanistan,2018,Economic status (wealth quintile),Quintile 2,2,Overweight,4.670275,3767.884277,4.112625
388,Eastern Mediterranean,AFG,Afghanistan,2018,Economic status (wealth quintile),Quintile 3,3,Overweight,4.078213,4178.454102,4.112625
389,Eastern Mediterranean,AFG,Afghanistan,2018,Economic status (wealth quintile),Quintile 4,4,Overweight,3.264172,4315.422852,4.112625
390,Eastern Mediterranean,AFG,Afghanistan,2018,Economic status (wealth quintile),Quintile 5 (richest),5,Overweight,4.254687,3575.646240,4.112625


In [509]:
# Check the new DataFrames
education_df.head()

,Region,Country ISO-3 Code,Country,Year,Dimension,Dimension Value,Dimension Value Order,Anthropometric Indicator,Prevalence Estimate %,Under-Five Population (million),Country Avg
391,Eastern Mediterranean,AFG,Afghanistan,2018,Education (mother),No education,1,Overweight,4.183403,16289.356445,4.112625
392,Eastern Mediterranean,AFG,Afghanistan,2018,Education (mother),Primary education,2,Overweight,2.141115,1341.238159,4.112625
393,Eastern Mediterranean,AFG,Afghanistan,2018,Education (mother),Secondary or higher education,3,Overweight,4.753134,1926.877563,4.112625
452,Eastern Mediterranean,AFG,Afghanistan,2018,Education (mother),No education,1,Stunting,40.559396,15851.871094,38.192776
453,Eastern Mediterranean,AFG,Afghanistan,2018,Education (mother),Primary education,2,Stunting,29.575229,1330.618286,38.192776


In [556]:
# Export dataframes as CSV files
age_df.to_csv("data/clean_data_by_age.csv", index=False)
sex_df.to_csv("data/clean_data_by_sex.csv", index=False)
residence_df.to_csv("data/clean_data_by_residence.csv", index=False)
economic_status_df.to_csv("data/clean_data_by_economic_status.csv", index=False)
education_df.to_csv("data/clean_data_by_education.csv", index=False)

## Normalize the data

In [511]:
# Create a new DataFrame with unique values from 'Country ISO-3 Code' and 'Country' columns
country_df = cleaned_df[['Country ISO-3 Code', 'Country']].drop_duplicates()
# Display the new DataFrame
print(country_df)


       Country ISO-3 Code                         Country
1                     AFG                     Afghanistan
1005                  ALB                         Albania
1555                  DZA                         Algeria
2600                  AGO                          Angola
3164                  ARG                       Argentina
...                   ...                             ...
144521                VNM                        Viet Nam
151646                YEM                           Yemen
153070                ZMB                          Zambia
154420                ZWE                        Zimbabwe
156020                PSE  occupied Palestinian territory

[154 rows x 2 columns]


In [512]:
# Create numpy array for each of the countries
country_ids = np.arange(1, len(country_df) + 1)
print(country_ids)

[  1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18
  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36
  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54
  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72
  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90
  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107 108
 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126
 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144
 145 146 147 148 149 150 151 152 153 154]


In [513]:
# Use a list comprehension to add "ctry" to each country_id.
ctry_ids = ["ctry" + str(ctry_id) for ctry_id in country_ids]

print(ctry_ids)

['ctry1', 'ctry2', 'ctry3', 'ctry4', 'ctry5', 'ctry6', 'ctry7', 'ctry8', 'ctry9', 'ctry10', 'ctry11', 'ctry12', 'ctry13', 'ctry14', 'ctry15', 'ctry16', 'ctry17', 'ctry18', 'ctry19', 'ctry20', 'ctry21', 'ctry22', 'ctry23', 'ctry24', 'ctry25', 'ctry26', 'ctry27', 'ctry28', 'ctry29', 'ctry30', 'ctry31', 'ctry32', 'ctry33', 'ctry34', 'ctry35', 'ctry36', 'ctry37', 'ctry38', 'ctry39', 'ctry40', 'ctry41', 'ctry42', 'ctry43', 'ctry44', 'ctry45', 'ctry46', 'ctry47', 'ctry48', 'ctry49', 'ctry50', 'ctry51', 'ctry52', 'ctry53', 'ctry54', 'ctry55', 'ctry56', 'ctry57', 'ctry58', 'ctry59', 'ctry60', 'ctry61', 'ctry62', 'ctry63', 'ctry64', 'ctry65', 'ctry66', 'ctry67', 'ctry68', 'ctry69', 'ctry70', 'ctry71', 'ctry72', 'ctry73', 'ctry74', 'ctry75', 'ctry76', 'ctry77', 'ctry78', 'ctry79', 'ctry80', 'ctry81', 'ctry82', 'ctry83', 'ctry84', 'ctry85', 'ctry86', 'ctry87', 'ctry88', 'ctry89', 'ctry90', 'ctry91', 'ctry92', 'ctry93', 'ctry94', 'ctry95', 'ctry96', 'ctry97', 'ctry98', 'ctry99', 'ctry100', 'ctry10

In [514]:
# Add the ctry_ids list as a new column
country_df['Country ID'] = ctry_ids
print(country_df)

       Country ISO-3 Code                         Country Country ID
1                     AFG                     Afghanistan      ctry1
1005                  ALB                         Albania      ctry2
1555                  DZA                         Algeria      ctry3
2600                  AGO                          Angola      ctry4
3164                  ARG                       Argentina      ctry5
...                   ...                             ...        ...
144521                VNM                        Viet Nam    ctry150
151646                YEM                           Yemen    ctry151
153070                ZMB                          Zambia    ctry152
154420                ZWE                        Zimbabwe    ctry153
156020                PSE  occupied Palestinian territory    ctry154

[154 rows x 3 columns]


In [515]:
# Reorder the columns so that 'Country ID' is first
country_df = country_df[['Country ID', 'Country ISO-3 Code', 'Country']]

# Display the updated DataFrame
country_df

,Country ID,Country ISO-3 Code,Country
1,ctry1,AFG,Afghanistan
1005,ctry2,ALB,Albania
1555,ctry3,DZA,Algeria
2600,ctry4,AGO,Angola
3164,ctry5,ARG,Argentina
...,...,...,...
144521,ctry150,VNM,Viet Nam
151646,ctry151,YEM,Yemen
153070,ctry152,ZMB,Zambia
154420,ctry153,ZWE,Zimbabwe


In [516]:
# Export country_df as CSV file.
country_df.to_csv("data/country.csv", index=False)

In [517]:
# Create a new DataFrame with unique values from 'Anthropometric Indicator' and 'Indicator Description' columns
indicator_df = cleaned_df[['Anthropometric Indicator']].drop_duplicates()

# Display the new DataFrame
print(indicator_df)

   Anthropometric Indicator
1                Overweight
21                 Stunting
41              Underweight
61                  Wasting
81           Wasting Severe


In [518]:
# Create numpy array for each of the Dimensions
indicator_ids = np.arange(1, len(indicator_df) + 1)
print(indicator_ids)

[1 2 3 4 5]


In [519]:
# Use a list comprehension to add "ind" to each ind_id.
ind_ids = ["ind" + str(ind_id) for ind_id in indicator_ids]

print(ind_ids)

['ind1', 'ind2', 'ind3', 'ind4', 'ind5']


In [520]:
# Add the ind_ids list as a new column
indicator_df['Indicator ID'] = ind_ids

print(indicator_df)

   Anthropometric Indicator Indicator ID
1                Overweight         ind1
21                 Stunting         ind2
41              Underweight         ind3
61                  Wasting         ind4
81           Wasting Severe         ind5


In [521]:
# Reorder the columns so that 'Indicator ID' is first
indicator_df = indicator_df[['Indicator ID', 'Anthropometric Indicator']]

# Display the updated DataFrame
indicator_df

,Indicator ID,Anthropometric Indicator
1,ind1,Overweight
21,ind2,Stunting
41,ind3,Underweight
61,ind4,Wasting
81,ind5,Wasting Severe


In [522]:
# Export indicator_df as CSV file.
indicator_df.to_csv("data/indicator.csv", index=False)

In [523]:
# Create a new DataFrame with unique values from the 'Dimension' column
dimension_df = cleaned_df[['Dimension']].drop_duplicates()

# Display the new DataFrame
print(dimension_df)

                             Dimension
1                         Age (months)
6                                  Sex
106                 Place of residence
386  Economic status (wealth quintile)
391                 Education (mother)


In [524]:
# Create numpy array for each of the Dimensions
dimension_ids = np.arange(1, len(dimension_df) + 1)
print(dimension_ids)

[1 2 3 4 5]


In [525]:
# Use a list comprehension to add "dim" to each dimension_id.
dim_ids = ["dim" + str(dim_id) for dim_id in dimension_ids]

print(dim_ids)

['dim1', 'dim2', 'dim3', 'dim4', 'dim5']


In [526]:
# Add the dimension_ids list as a new column
dimension_df['Dimension ID'] = dim_ids

print(dimension_df)

                             Dimension Dimension ID
1                         Age (months)         dim1
6                                  Sex         dim2
106                 Place of residence         dim3
386  Economic status (wealth quintile)         dim4
391                 Education (mother)         dim5


In [527]:
# Reorder the columns so that 'Dimension ID' is first, then 'Dimension'
dimension_df = dimension_df[['Dimension ID', 'Dimension']]

# Display the updated DataFrame
dimension_df

,Dimension ID,Dimension
1,dim1,Age (months)
6,dim2,Sex
106,dim3,Place of residence
386,dim4,Economic status (wealth quintile)
391,dim5,Education (mother)


In [528]:
# Export dimension_df as CSV file.
dimension_df.to_csv("data/dimension.csv", index=False)

In [529]:
# Create a new DataFrame with unique values from the 'Dimension Value' column
dimension_value_df = cleaned_df[['Dimension Value']].drop_duplicates()

# Display the new DataFrame
print(dimension_value_df)

                   Dimension Value
1                     12-23 months
2                     24-35 months
3                     36-47 months
4                     48-59 months
5                      6-11 months
6                           Female
7                             Male
100                     0-5 months
106                          Rural
107                          Urban
386           Quintile 1 (poorest)
387                     Quintile 2
388                     Quintile 3
389                     Quintile 4
390           Quintile 5 (richest)
391                   No education
392              Primary education
393  Secondary or higher education


In [530]:
# Create numpy array for each of the dimension values
dimension_value_ids = np.arange(1, len(dimension_value_df) + 1)
print(dimension_value_ids)

[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18]


In [531]:
# Use a list comprehension to add "dimval" to each dimension_id.
dimval_ids = ["dimval" + str(dimval_id) for dimval_id in dimension_value_ids]

print(dimval_ids)

['dimval1', 'dimval2', 'dimval3', 'dimval4', 'dimval5', 'dimval6', 'dimval7', 'dimval8', 'dimval9', 'dimval10', 'dimval11', 'dimval12', 'dimval13', 'dimval14', 'dimval15', 'dimval16', 'dimval17', 'dimval18']


In [532]:
# Add the dimval_ids list as a new column
dimension_value_df['Dimension Value ID'] = dimval_ids

print(dimension_value_df)

                   Dimension Value Dimension Value ID
1                     12-23 months            dimval1
2                     24-35 months            dimval2
3                     36-47 months            dimval3
4                     48-59 months            dimval4
5                      6-11 months            dimval5
6                           Female            dimval6
7                             Male            dimval7
100                     0-5 months            dimval8
106                          Rural            dimval9
107                          Urban           dimval10
386           Quintile 1 (poorest)           dimval11
387                     Quintile 2           dimval12
388                     Quintile 3           dimval13
389                     Quintile 4           dimval14
390           Quintile 5 (richest)           dimval15
391                   No education           dimval16
392              Primary education           dimval17
393  Secondary or higher edu

In [533]:
# Reorder the columns so that 'Dimension Value ID' is first
dimension_value_df = dimension_value_df[['Dimension Value ID', 'Dimension Value']]

# Display the updated DataFrame
dimension_value_df

,Dimension Value ID,Dimension Value
1,dimval1,12-23 months
2,dimval2,24-35 months
3,dimval3,36-47 months
4,dimval4,48-59 months
5,dimval5,6-11 months
6,dimval6,Female
7,dimval7,Male
100,dimval8,0-5 months
106,dimval9,Rural
107,dimval10,Urban


In [534]:
# Export dimension_value_df as CSV file.
dimension_value_df.to_csv("data/dimension_value.csv", index=False)

In [535]:
# Create a new DataFrame with unique values from the 'Region' column
region_df = cleaned_df[['Region']].drop_duplicates()

# Display the new DataFrame
print(region_df)

                     Region
1     Eastern Mediterranean
1005               European
1555                African
3164               Americas
4525        Western Pacific
5308        South-East Asia


In [536]:
# Create numpy array for each of the regions
region_ids = np.arange(1, len(region_df) + 1)
print(region_ids)

[1 2 3 4 5 6]


In [537]:
# Use a list comprehension to add "reg" to each region_ids.
reg_ids = ["reg" + str(reg_id) for reg_id in region_ids]

print(reg_ids)

['reg1', 'reg2', 'reg3', 'reg4', 'reg5', 'reg6']


In [538]:
# Add the reg_ids list as a new column
region_df['Region ID'] = reg_ids

print(region_df)

                     Region Region ID
1     Eastern Mediterranean      reg1
1005               European      reg2
1555                African      reg3
3164               Americas      reg4
4525        Western Pacific      reg5
5308        South-East Asia      reg6


In [539]:
# Reorder the columns so that 'Region ID' is first
region_df = region_df[['Region ID', 'Region']]

# Display the updated DataFrame
region_df

,Region ID,Region
1,reg1,Eastern Mediterranean
1005,reg2,European
1555,reg3,African
3164,reg4,Americas
4525,reg5,Western Pacific
5308,reg6,South-East Asia


In [540]:
# Export region_df as CSV file.
region_df.to_csv("data/region.csv", index=False)

## Data loading

In [ ]:
import sqlite3

# Establish a SQLite connection
conn = sqlite3.connect('data/malnutrition_data.db')

In [560]:
# Save the cleaned data set to the SQLite database
cleaned_df.to_sql('clean_data', conn, if_exists='replace', index=False)

# Query to ensure the data was stored correctly
df_from_db = pd.read_sql('SELECT * FROM clean_data', conn)
df_from_db.head()

,Region,Country ISO-3 Code,Country,Year,Dimension,Dimension Value,Dimension Value Order,Anthropometric Indicator,Prevalence Estimate %,Under-Five Population (million),Country Avg
0,Eastern Mediterranean,AFG,Afghanistan,2004,Age (months),12-23 months,3,Overweight,4.3,189.0,4.6
1,Eastern Mediterranean,AFG,Afghanistan,2004,Age (months),24-35 months,4,Overweight,3.0,182.0,4.6
2,Eastern Mediterranean,AFG,Afghanistan,2004,Age (months),36-47 months,5,Overweight,5.6,227.0,4.6
3,Eastern Mediterranean,AFG,Afghanistan,2004,Age (months),48-59 months,6,Overweight,6.4,241.0,4.6
4,Eastern Mediterranean,AFG,Afghanistan,2004,Age (months),6-11 months,2,Overweight,1.9,107.0,4.6


In [561]:
# Save the country data set to the SQLite database
country_df.to_sql('country', conn, if_exists='replace', index=False)

# Query to ensure the data was stored correctly
df_from_db = pd.read_sql('SELECT * FROM country', conn)
df_from_db.head()

,Country ID,Country ISO-3 Code,Country
0,ctry1,AFG,Afghanistan
1,ctry2,ALB,Albania
2,ctry3,DZA,Algeria
3,ctry4,AGO,Angola
4,ctry5,ARG,Argentina


In [562]:
# Save the dimension value data set to the SQLite database
dimension_value_df.to_sql('dimension_value', conn, if_exists='replace', index=False)

# Query to ensure the data was stored correctly
df_from_db = pd.read_sql('SELECT * FROM dimension_value', conn)
df_from_db.head()

,Dimension Value ID,Dimension Value
0,dimval1,12-23 months
1,dimval2,24-35 months
2,dimval3,36-47 months
3,dimval4,48-59 months
4,dimval5,6-11 months


In [563]:
# Save the dimension data set to the SQLite database
dimension_df.to_sql('dimension', conn, if_exists='replace', index=False)

# Query to ensure the data was stored correctly
df_from_db = pd.read_sql('SELECT * FROM dimension', conn)
df_from_db.head()

,Dimension ID,Dimension
0,dim1,Age (months)
1,dim2,Sex
2,dim3,Place of residence
3,dim4,Economic status (wealth quintile)
4,dim5,Education (mother)


In [564]:
# Save the indicator data set to the SQLite database
indicator_df.to_sql('indicator', conn, if_exists='replace', index=False)

# Query to ensure the data was stored correctly
df_from_db = pd.read_sql('SELECT * FROM indicator', conn)
df_from_db.head()

,Indicator ID,Anthropometric Indicator
0,ind1,Overweight
1,ind2,Stunting
2,ind3,Underweight
3,ind4,Wasting
4,ind5,Wasting Severe


In [557]:
# Save the region dataset to the SQLite database
region_df.to_sql('region', conn, if_exists='replace', index=False)

# Query to ensure the data was stored correctly
df_from_db = pd.read_sql('SELECT * FROM region', conn)
df_from_db.head()

,Region ID,Region
0,reg1,Eastern Mediterranean
1,reg2,European
2,reg3,African
3,reg4,Americas
4,reg5,Western Pacific


In [559]:
# Save the dimension dataset to the SQLite database
dimension_df.to_sql('dimension', conn, if_exists='replace', index=False)

# Query to ensure the data was stored correctly
df_from_db = pd.read_sql('SELECT * FROM dimension', conn)
df_from_db.head()

,Dimension ID,Dimension
0,dim1,Age (months)
1,dim2,Sex
2,dim3,Place of residence
3,dim4,Economic status (wealth quintile)
4,dim5,Education (mother)


## Data extraction from SQLite database

In [568]:
# Query the database from the clean data table by the age dimension and create a dataframe
age_df_from_db = pd.read_sql("SELECT * FROM clean_data WHERE Dimension == 'Age (months)'", conn)
age_df_from_db.head()

,Region,Country ISO-3 Code,Country,Year,Dimension,Dimension Value,Dimension Value Order,Anthropometric Indicator,Prevalence Estimate %,Under-Five Population (million),Country Avg
0,Eastern Mediterranean,AFG,Afghanistan,2004,Age (months),12-23 months,3,Overweight,4.3,189.0,4.6
1,Eastern Mediterranean,AFG,Afghanistan,2004,Age (months),24-35 months,4,Overweight,3.0,182.0,4.6
2,Eastern Mediterranean,AFG,Afghanistan,2004,Age (months),36-47 months,5,Overweight,5.6,227.0,4.6
3,Eastern Mediterranean,AFG,Afghanistan,2004,Age (months),48-59 months,6,Overweight,6.4,241.0,4.6
4,Eastern Mediterranean,AFG,Afghanistan,2004,Age (months),6-11 months,2,Overweight,1.9,107.0,4.6


In [570]:
dimension_df

,Dimension ID,Dimension
1,dim1,Age (months)
6,dim2,Sex
106,dim3,Place of residence
386,dim4,Economic status (wealth quintile)
391,dim5,Education (mother)


In [577]:
# Query the database from the clean data table by age and create a dataframe
economic_status_df_from_db = pd.read_sql("SELECT * FROM clean_data WHERE Dimension == 'Economic status (wealth quintile)'", conn)
economic_status_df_from_db.head()

,Region,Country ISO-3 Code,Country,Year,Dimension,Dimension Value,Dimension Value Order,Anthropometric Indicator,Prevalence Estimate %,Under-Five Population (million),Country Avg
0,Eastern Mediterranean,AFG,Afghanistan,2018,Economic status (wealth quintile),Quintile 1 (poorest),1,Overweight,4.486930,3576.301514,4.112625
1,Eastern Mediterranean,AFG,Afghanistan,2018,Economic status (wealth quintile),Quintile 2,2,Overweight,4.670275,3767.884277,4.112625
2,Eastern Mediterranean,AFG,Afghanistan,2018,Economic status (wealth quintile),Quintile 3,3,Overweight,4.078213,4178.454102,4.112625
3,Eastern Mediterranean,AFG,Afghanistan,2018,Economic status (wealth quintile),Quintile 4,4,Overweight,3.264172,4315.422852,4.112625
4,Eastern Mediterranean,AFG,Afghanistan,2018,Economic status (wealth quintile),Quintile 5 (richest),5,Overweight,4.254687,3575.646240,4.112625


In [578]:
# Query the database from the clean data table by economic status and create a dataframe
education_df_from_db = pd.read_sql("SELECT * FROM clean_data WHERE Dimension == 'Education (mother)'", conn)
education_df_from_db.head()

,Region,Country ISO-3 Code,Country,Year,Dimension,Dimension Value,Dimension Value Order,Anthropometric Indicator,Prevalence Estimate %,Under-Five Population (million),Country Avg
0,Eastern Mediterranean,AFG,Afghanistan,2018,Education (mother),No education,1,Overweight,4.183403,16289.356445,4.112625
1,Eastern Mediterranean,AFG,Afghanistan,2018,Education (mother),Primary education,2,Overweight,2.141115,1341.238159,4.112625
2,Eastern Mediterranean,AFG,Afghanistan,2018,Education (mother),Secondary or higher education,3,Overweight,4.753134,1926.877563,4.112625
3,Eastern Mediterranean,AFG,Afghanistan,2018,Education (mother),No education,1,Stunting,40.559396,15851.871094,38.192776
4,Eastern Mediterranean,AFG,Afghanistan,2018,Education (mother),Primary education,2,Stunting,29.575229,1330.618286,38.192776


In [579]:
# Query the database from the clean data table by place of residence and create a dataframe
residence_df_from_db = pd.read_sql("SELECT * FROM clean_data WHERE Dimension == 'Place of residence'", conn)
residence_df_from_db.head()

,Region,Country ISO-3 Code,Country,Year,Dimension,Dimension Value,Dimension Value Order,Anthropometric Indicator,Prevalence Estimate %,Under-Five Population (million),Country Avg
0,Eastern Mediterranean,AFG,Afghanistan,2013,Place of residence,Rural,0,Overweight,5.969128,3.234018e+06,5.341543
1,Eastern Mediterranean,AFG,Afghanistan,2013,Place of residence,Urban,0,Overweight,3.338518,1.013278e+06,5.341543
2,Eastern Mediterranean,AFG,Afghanistan,2013,Place of residence,Rural,0,Stunting,43.330780,3.214813e+06,40.413544
3,Eastern Mediterranean,AFG,Afghanistan,2013,Place of residence,Urban,0,Stunting,31.130192,1.010235e+06,40.413544
4,Eastern Mediterranean,AFG,Afghanistan,2013,Place of residence,Rural,0,Underweight,26.375696,3.373931e+06,24.643473


In [580]:
# Query the database from the clean data table by sex and create a dataframe
sex_df_from_db = pd.read_sql("SELECT * FROM clean_data WHERE Dimension == 'Sex'", conn)
sex_df_from_db.head()

,Region,Country ISO-3 Code,Country,Year,Dimension,Dimension Value,Dimension Value Order,Anthropometric Indicator,Prevalence Estimate %,Under-Five Population (million),Country Avg
0,Eastern Mediterranean,AFG,Afghanistan,2004,Sex,Female,0,Overweight,2.6,436.0,4.6
1,Eastern Mediterranean,AFG,Afghanistan,2004,Sex,Male,0,Overweight,6.3,510.0,4.6
2,Eastern Mediterranean,AFG,Afghanistan,2004,Sex,Female,0,Stunting,58.5,436.0,59.3
3,Eastern Mediterranean,AFG,Afghanistan,2004,Sex,Male,0,Stunting,60.0,510.0,59.3
4,Eastern Mediterranean,AFG,Afghanistan,2004,Sex,Female,0,Underweight,33.0,436.0,32.9


In [581]:
# Export the dataframes to CSV files
age_df_from_db.to_csv("data/clean_data_by_age_from_db.csv", index=False)
economic_status_df_from_db.to_csv("data/clean_data_by_economic_status_from_db.csv", index=False)
education_df_from_db.to_csv("data/clean_data_by_education_from_db.csv", index=False)
residence_df_from_db.to_csv("data/clean_data_by_residence_from_db.csv", index=False)
sex_df_from_db.to_csv("data/clean_data_by_sex_from_db.csv", index=False)